# Loop 65 Analysis: CV-LB Relationship and Submission Decision

Major breakthrough: exp_069 achieved CV = 0.005146 (35% improvement over previous best 0.007938)

Key question: Does this fundamentally different approach (CatBoost + XGBoost) have a different CV-LB relationship?

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

# All submissions with CV and LB scores
submissions = [
    {'exp': 'exp_000', 'CV': 0.0111, 'LB': 0.0982, 'model': 'MLP'},
    {'exp': 'exp_001', 'CV': 0.0123, 'LB': 0.1065, 'model': 'LGBM'},
    {'exp': 'exp_003', 'CV': 0.0105, 'LB': 0.0972, 'model': 'MLP'},
    {'exp': 'exp_005', 'CV': 0.0104, 'LB': 0.0969, 'model': 'MLP'},
    {'exp': 'exp_006', 'CV': 0.0097, 'LB': 0.0946, 'model': 'MLP'},
    {'exp': 'exp_007', 'CV': 0.0093, 'LB': 0.0932, 'model': 'MLP'},
    {'exp': 'exp_009', 'CV': 0.0092, 'LB': 0.0936, 'model': 'Ridge'},
    {'exp': 'exp_012', 'CV': 0.0090, 'LB': 0.0913, 'model': 'MLP+GP'},
    {'exp': 'exp_024', 'CV': 0.0087, 'LB': 0.0893, 'model': 'MLP+GP'},
    {'exp': 'exp_026', 'CV': 0.0085, 'LB': 0.0887, 'model': 'MLP+GP'},
    {'exp': 'exp_030', 'CV': 0.0083, 'LB': 0.0877, 'model': 'MLP+GP'},
    {'exp': 'exp_041', 'CV': 0.0090, 'LB': 0.0932, 'model': 'XGB'},
    {'exp': 'exp_042', 'CV': 0.0145, 'LB': 0.1147, 'model': 'GroupKFold'},
]

df = pd.DataFrame(submissions)
print('Submission history:')
print(df.to_string(index=False))
print(f'\nBest LB: {df["LB"].min():.4f} (exp_030)')
print(f'Best CV: {df["CV"].min():.4f} (exp_030)')

In [ ]:
# Fit linear regression: LB = slope * CV + intercept
slope, intercept, r_value, p_value, std_err = stats.linregress(df['CV'], df['LB'])

print(f'CV-LB Linear Fit:')
print(f'  LB = {slope:.3f} * CV + {intercept:.4f}')
print(f'  R² = {r_value**2:.4f}')
print(f'  Intercept = {intercept:.4f}')
print(f'\nInterpretation:')
print(f'  - Even at CV=0, expected LB would be {intercept:.4f}')
print(f'  - Target is 0.0347')
print(f'  - Gap: {intercept - 0.0347:.4f}')

# Predict LB for new CV = 0.005146
new_cv = 0.005146
predicted_lb = slope * new_cv + intercept
print(f'\nPrediction for exp_069 (CV = {new_cv}):')
print(f'  Predicted LB = {slope:.3f} * {new_cv} + {intercept:.4f} = {predicted_lb:.4f}')
print(f'  This would be the BEST LB if the relationship holds!')

In [ ]:
# Key strategic question: Does CatBoost+XGBoost have a DIFFERENT CV-LB relationship?

print('='*70)
print('STRATEGIC ANALYSIS')
print('='*70)

print('\n1. CURRENT SITUATION:')
print(f'   - Best CV: 0.005146 (exp_069, CatBoost+XGBoost)')
print(f'   - Best LB: 0.0877 (exp_030, MLP+GP)')
print(f'   - Target: 0.0347')

print('\n2. CV-LB RELATIONSHIP (from 13 submissions):')
print(f'   - LB = {slope:.3f} * CV + {intercept:.4f} (R² = {r_value**2:.4f})')
print(f'   - Intercept ({intercept:.4f}) > Target (0.0347)')
print(f'   - This means: even with CV=0, LB would be ~{intercept:.4f}')

print('\n3. PREDICTION FOR exp_069:')
print(f'   - If relationship holds: LB = {predicted_lb:.4f}')
print(f'   - This would be BEST LB (vs current best 0.0877)')
print(f'   - But still far from target (0.0347)')

print('\n4. KEY QUESTION:')
print('   Does CatBoost+XGBoost have a DIFFERENT CV-LB relationship?')
print('   - Previous submissions: MLP, LGBM, Ridge, GP, XGB')
print('   - exp_069: CatBoost + XGBoost with feature priority filtering')
print('   - This is fundamentally different (gradient boosting ensemble)')
print('   - The only way to know is to SUBMIT and check!')

print('\n5. SUBMISSION DECISION:')
print('   - 5 submissions remaining')
print('   - exp_069 has 35% better CV than any previous submission')
print('   - MUST SUBMIT to verify if CV-LB relationship changes')
print('   - Even if relationship holds, predicted LB (0.075) would be best')

In [ ]:
# What CV would be needed to reach target?

print('='*70)
print('REQUIRED CV TO REACH TARGET')
print('='*70)

target = 0.0347

# Using current linear fit
required_cv = (target - intercept) / slope
print(f'\nUsing current linear fit (LB = {slope:.3f}*CV + {intercept:.4f}):')
print(f'  Required CV = (0.0347 - {intercept:.4f}) / {slope:.3f} = {required_cv:.6f}')

if required_cv < 0:
    print(f'  IMPOSSIBLE: Required CV is NEGATIVE!')
    print(f'  The intercept ({intercept:.4f}) is already above target (0.0347)')
    print(f'  We need to CHANGE THE RELATIONSHIP, not just improve CV')
else:
    print(f'  Required CV: {required_cv:.6f}')
    print(f'  Current best CV: 0.005146')
    print(f'  Gap: {0.005146 - required_cv:.6f}')

print('\n' + '='*70)
print('CRITICAL INSIGHT')
print('='*70)
print('\nThe intercept (0.0534) > target (0.0347) means:')
print('  - We CANNOT reach target by improving CV alone')
print('  - We need approaches that CHANGE the CV-LB relationship')
print('  - CatBoost+XGBoost MAY have a different relationship!')
print('  - SUBMIT exp_069 to test this hypothesis!')